In [1]:
from scripts.software_frs import frs_traceability2
from officelib.xllib import *
from officelib.const import xlconst as xlc
import sys
import os

In [2]:
# caching scheme for test purposes. 
try:
    _cache
except NameError:
    _cache = None

try:
    _lffi
except NameError:
    _lffi = frs_traceability2.load_frs_from_issuetracker
    
def load_frs_from_issuetracker():
    global _cache
    if _cache is None:
        frs = _lffi()
        _cache = frs
    else:
        frs = _cache
    return frs

frs_traceability2.load_frs_from_issuetracker = load_frs_from_issuetracker

In [3]:
def ws_data(ws):
    cr = ws.Cells.Range
    first = cr("A2")
    last_col = cr("A1").End(xlc.xlToRight).Column
    last = first.End(xlc.xlDown).Offset(1, last_col)
    return cr(first, last).Value

def split_frs(frs):
    return [f for f in frs.split() if f]

def load_test_list(user_tests):
    xl = Excel(visible=False)
    with HiddenXl(xl):
        wb = xl.Workbooks.Open(user_tests)
        ws = wb.Worksheets(1)
        id_test, name, purpose, start, steps, accept, frs = \
                        frs_traceability2.find_cols(ws, 
                            "ID_TEST", "NAME", "PURPOSE", "STARTING POINT", 
                            "STEPS", "ACCEPTANCE_CRITERIA", "List Web FRS")
        data = ws_data(ws)
    
    wb.Close(False)
    xl.Quit()
    
    res = []
    for row in data:
        i = row[id_test]
        n = row[name]
        p = row[purpose]
        sta = row[start]
        ste = row[steps]
        a = row[accept]
        f = row[frs]
        if not f: continue
        res.append((i, n, p, sta, ste, a, split_frs(f)))
    res.sort(key=lambda t: t[0])
    return res

In [4]:
def node_data(node):
    return node.id, node.text.strip()

def make_frs_line(items):
    lines = []
    items2 = sorted(items, key=frs_traceability2.sort_frs_item)
    for id, line in items2:
        if not line:
            lines.append("%s" % id)
        else:
            lines.append("%s: %s" % (id, line.rstrip()))
    return "\n".join(lines)

def ifmt(i):
    if int(i) == i:
        return "%d" % i
    return "%s" % i

def write_test(reqs, test, file):
    tmplt = _frs_tmplt
    i, name, p, sta, ste, a, frs = test
    items = set()
    for f in frs:
        node = reqs.lookup(f)
        if not node: raise ValueError(f)
        if node.is_na(): continue
        items.add((node.id, node.text))
        indent = node.id.count(".")
        for n in node.iter():
            nindent = n.id.count(".") - indent
            items.add((" " * nindent + n.id, n.text))
    frs_line = make_frs_line(items)
    s = tmplt % (ifmt(i), name, frs_line, p.rstrip(), sta.rstrip(), ste.rstrip(), a.rstrip())
    print(s, file=file)

In [5]:
_c = "cache.pkl"
import os, pickle
def download_relevant_issues():
    global _c
    if os.path.exists(_c):
        return pickle.load(open(_c, 'rb'))
    else:
        d = _dl()
        pickle.dump(d, open(_c, 'wb'))
        return d
try:
    _dl
except NameError:
    _dl = frs_traceability2.download_relevant_issues
    frs_traceability2.download_relevant_issues = download_relevant_issues

AttributeError: module 'scripts.software_frs.frs_traceability2' has no attribute 'download_relevant_issues'

In [6]:
try:
    _reqs
except NameError:
    _reqs = None

def main(user_tests, unit_tests, code_reviews, fp=None):
    global _reqs
    reqs = _reqs or frs_traceability2.full_frs_tree(user_tests, unit_tests, code_reviews)
    _reqs = reqs
    test_list = load_test_list(user_tests)
    if fp is None:
        f = sys.stdout
        close = False
    else:
        f = open(fp, 'w')
        close = True
    try:
        for test in test_list: write_test(reqs, test, f)
    finally:
        if close:
            f.close()
    

In [7]:
trace_path = "C:\\Users\\Nathan\\Documents\\PBS\\SW test\\3.0 User Test\\Traceability"
user_tests = "Hello tests 161122 1.xlsx"
unit_tests = "Traceability Matrix (Reqs and VI) 161123.csv"
code_review_path = '\\\\PBSSTATION\\PBSCloudShare\\(2) R&D-Product Engineering\\Software Development\\3.0 Project\\Phase 2 Working Copy\\IS00038B6 Rev A Code Reviews\\'
p1 = os.path.join(trace_path, user_tests)
p2 = os.path.join(trace_path, unit_tests)
p3 = code_review_path

In [8]:
_frs_tmplt = \
"""
=============================================
Id: %s
Name: %s
=============================================
FRS:
%s

Purpose: %s

Starting Point:
%s

Steps:
%s

Acceptance Criteria:
%s
"""

In [9]:
from datetime import date
def dateify(f):
    a, b = f.split(".")
    t = date.today()
    a += "_" + t.strftime("%y%m%d")
    return ".".join((a,b))

In [10]:
_reqs = None
#os.remove(_relcache)
f = "frs.txt"
f = dateify(f)
main(p1, p2, p3, f)
os.startfile(f)

Loading User Test Matrix...
Loading Unit Test Matrix...
Loading Code Review Matrix...


In [11]:
main(p1, p2, p3)

Loading User Test Matrix...
Loading Unit Test Matrix...
Loading Code Review Matrix...


TypeError: This COM object can not automate the makepy process - please run makepy manually for this object